In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib as mplt
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from IPython.core.interactiveshell import InteractiveShell
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import explained_variance_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
%matplotlib inline
import seaborn as sns
import math
import gc
import ipaddress
from urllib.parse import urlparse
from tldextract import extract
from sklearn.metrics import confusion_matrix

import sys
from pathlib import Path
d = Path().resolve().parent.parent
sys.path.insert(0, str(d))
import util.utils as utils
import util.plot_utils as plot_utils

from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import lightgbm as lgb

np.set_printoptions(threshold=np.nan)



plt.rcParams["figure.figsize"] = (12,4)

from xgboost import XGBClassifier
import xgboost as xgb
from sklearn.metrics import accuracy_score


InteractiveShell.ast_node_interactivity = "all"

In [2]:
df_hpg_reserve = pd.read_csv("data/hpg_reserve.csv")
df_air_reserve = pd.read_csv("data/air_reserve.csv")

In [3]:
df_air_store_info = pd.read_csv("data/air_store_info.csv")
df_hpg_store_info = pd.read_csv("data/hpg_store_info.csv")
df_store_id_relation = pd.read_csv("data/store_id_relation.csv")
df_air_visit_data = pd.read_csv("data/air_visit_data.csv")
df_date_info = pd.read_csv("data/date_info.csv")

#### Are All Air Ids to predict present in 
- air visit data - Yes
- air store info - Yes
- air reserve - No

#### Are All Air visit data in air store info: Yes
- Is vice versa true - Yes


In [4]:
sample_sub = pd.read_csv("data/sample_submission.csv")

pred_ids = sample_sub.id.map(lambda x:x[:-11])
pred_ids[~pred_ids.isin(df_air_visit_data["air_store_id"])]
pred_ids[~pred_ids.isin(df_air_store_info["air_store_id"])]
pred_ids[~pred_ids.isin(df_air_reserve["air_store_id"])].nunique()

air_visit_ids = df_air_visit_data["air_store_id"]
air_visit_ids[~air_visit_ids.isin(df_air_store_info["air_store_id"])]

df_air_store_info["air_store_id"][~df_air_store_info["air_store_id"].isin(df_air_visit_data["air_store_id"])]


Series([], Name: id, dtype: object)

Series([], Name: id, dtype: object)

507

Series([], Name: air_store_id, dtype: object)

Series([], Name: air_store_id, dtype: object)

In [5]:
df_air_visit_data["air_store_id"].nunique()

829

In [9]:
def loss(Y,A):
    length = len(Y)
    return np.sqrt(np.sum(np.square(np.log1p(Y) - np.log1p(A)))/length)



In [10]:
df_air_mean_visits = df_air_visit_data.groupby(["air_store_id"],as_index=False).mean()
loss(df_air_visit_data["visitors"],df_air_mean_visits["visitors"])

0.067217886813579056

In [49]:
df_air_mean_visits.head()
df_air_mean_visits.nunique()

,air_store_id,visitors
0,air_00a91d42b08b08d9,26.081897
1,air_0164b9927d20bcc3,9.248322
2,air_0241aa3964b7f861,9.896465
3,air_0328696196e46f18,7.939655
4,air_034a3d5b40d5b1b1,14.828685


air_store_id    829
visitors        828
dtype: int64

In [65]:
preds = pred_ids.map(lambda store:df_air_mean_visits[df_air_mean_visits["air_store_id"]==store]["visitors"].values[0])

In [67]:
sample_sub["visitors"] = preds

In [69]:
sample_sub.to_csv("results/simple-mean-pred_1.csv",index=False,columns=["id","visitors"])